In [1]:
import pandas as pd
import numpy as np
import gc

buro_bal = pd.read_pickle("buro_bal.p")
bureau   = pd.read_pickle("bureau.p")

In [2]:
ID = bureau['SK_ID_CURR'].unique()
data_base = pd.DataFrame()
data_base['SK_ID_CURR'] = ID

In [3]:
#Write Off Data
write_off = buro_bal[buro_bal['STATUS']=='5']
write_off = write_off.sort_values(['SK_ID_BUREAU','MONTHS_BALANCE'])
write_off = write_off.drop_duplicates(['SK_ID_BUREAU'],keep='last')

In [4]:
write_off = pd.merge(write_off,bureau[['SK_ID_BUREAU','SK_ID_CURR','CREDIT_TYPE_OVERALL','AMT_CREDIT_SUM','AMT_CREDIT_MAX_OVERDUE','AMT_CREDIT_SUM_DEBT','AMT_CREDIT_SUM_OVERDUE','CREDIT_TYPE']],on = 'SK_ID_BUREAU',how='inner')

In [5]:
def write_off_count(filter_type=None,data=None,original_data=None,new_varname = None):
    if filter_type is not None :
        temp_series = original_data[(original_data["CREDIT_TYPE_OVERALL"] == filter_type)].groupby(["SK_ID_CURR"]).count().SK_ID_BUREAU
        data_count  = pd.DataFrame()
        data_count  = pd.DataFrame({'SK_ID_CURR':temp_series.index, new_varname:temp_series.values})
        Summerized_data = data.merge(data_count,how="left")
    else :
        temp_series = original_data.groupby(["SK_ID_CURR"]).count().SK_ID_BUREAU
        data_count  = pd.DataFrame()
        data_count  = pd.DataFrame({'SK_ID_CURR':temp_series.index, new_varname:temp_series.values})
        Summerized_data = data.merge(data_count,how="left")    
    return Summerized_data

In [6]:
data_base = write_off_count(filter_type="Unsecured",data=data_base,original_data = write_off,new_varname= 'Unsecured_write_off_count')
data_base = write_off_count(filter_type="Secured", data=data_base,original_data = write_off,new_varname= 'Secured_write_off_count')
data_base = write_off_count(filter_type="HL",data=data_base,original_data = bureau,new_varname= 'HL_write_off_count')
data_base = write_off_count(data=data_base,original_data = bureau,new_varname= 'Total_write_off_count')

In [7]:
import numpy as np
Unsecured_percentile = np.percentile(bureau[bureau['CREDIT_TYPE_OVERALL']=='Unsecured']['AMT_CREDIT_SUM'], np.arange(0, 100, 10)) 
Secured_percentile   = np.percentile(bureau[bureau['CREDIT_TYPE_OVERALL']=='Secured']['AMT_CREDIT_SUM'], np.arange(0, 100, 10)) # quartiles
HL_percentile        = np.percentile(bureau[bureau['CREDIT_TYPE_OVERALL']=='HL']['AMT_CREDIT_SUM'], np.arange(0, 100, 10)) # quartiles
Total_percentile     = np.percentile(bureau['AMT_CREDIT_SUM'], np.arange(0, 100, 10))

In [8]:
def percentile(x,compare):
    if (x <= compare[1]):
        val = "1"
    elif(x  <= compare[2]):
        val = "2"
    elif  x  <= compare[3]:
        val = "3"
    elif  x  <= compare[4]:
        val = "4"  
    elif  x  <= compare[5]:
        val = "5"
    elif  x  <= compare[6]:
        val = "6"
    elif  x  <= compare[7]:
        val = "7"
    elif  x  <= compare[8]:
        val = "8" 
    elif  x  <= compare[9]:
        val = "9"     
    else:
        val = "10"
    return val

In [9]:
bureau['Unsecured_HCR_Tag'] = bureau[bureau['CREDIT_TYPE_OVERALL']=='Unsecured']['AMT_CREDIT_SUM'].apply(lambda x: percentile(x, Unsecured_percentile))
bureau['Secured_HCR_Tag']   = bureau[bureau['CREDIT_TYPE_OVERALL']=='Secured']['AMT_CREDIT_SUM'].apply(lambda x: percentile(x, Secured_percentile))
bureau['HL_HCR_Tag']        = bureau[bureau['CREDIT_TYPE_OVERALL']=='HL']['AMT_CREDIT_SUM'].apply(lambda x: percentile(x, HL_percentile))
bureau['Total_HCR_Tag']     = bureau['AMT_CREDIT_SUM'].apply(lambda x: percentile(x, Total_percentile))

In [10]:
bureau = pd.get_dummies(bureau,columns=['Unsecured_HCR_Tag'])

In [11]:
bureau = pd.get_dummies(bureau,columns=['Secured_HCR_Tag'])
bureau = pd.get_dummies(bureau,columns=['HL_HCR_Tag'])
bureau = pd.get_dummies(bureau,columns=['Total_HCR_Tag'])

In [12]:
def HCR_trade(column = None,data=None,original_data=None,new_varname = None):
    temp_series = original_data.groupby(["SK_ID_CURR"])[column].sum()
    data_count  = pd.DataFrame()
    data_count  = pd.DataFrame({'SK_ID_CURR':temp_series.index, new_varname:temp_series.values})
    Summerized_data = data.merge(data_count,how="left")
    return Summerized_data

In [13]:
columns = list(bureau.columns.values)
col = [col for col in columns if 'Tag' in col]

In [14]:
n = len(col)

for i in range(n):
    data_base = HCR_trade(column = col[i],data = data_base,original_data = bureau,new_varname=col[i])

In [15]:
columns_base = list(data_base.columns.values)

col_delq = [col for col in columns_base if 'delq' in col]
col_time = [col for col in columns_base if 'time' in col]
col_tag  = [col for col in columns_base if 'Tag' in col]
col_count  = [col for col in columns_base if 'count' in col]

In [16]:
for c in col_delq:
    data_base[c] = data_base[c].fillna(-99999)
    
for c in col_time:
    data_base[c] = data_base[c].fillna(-99999)
    
for c in col_tag:
    data_base[c] = data_base[c].fillna(0)
    
for c in col_count:
    data_base[c] = data_base[c].fillna(0)    

In [17]:
data_base.to_pickle("bureau_2.p")